# Chord Recognition


## Setting up Environment
```bash
$ python -m venv chord
$ source chord/bin/activate # for linux
$ ./chord/Scripts/activate # for windows
$ python.exe -m pip install --upgrade pip # upgrade pip
$ pip install -r requirements.txt
```

In [ ]:
# Import dependencies
import numpy as np
import matplotlib.pyplot as plt
import scipy
import librosa

In [ ]:
# Load all audio files in folder ./res/audio
audio_files = librosa.util.find_files('res/audio')
audio_files = np.asarray(audio_files)